## The Best Classifier Notebook

Importing the data into a pandas dataframe

In [63]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_7497bda9f8cf4c90afde8a7070a8ad28 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='zcT3AdF8Y9Qu7rbnFcEmeUOG41lPCbQv6yEJY_YPe74S',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_7497bda9f8cf4c90afde8a7070a8ad28.get_object(Bucket='thebestclassifier-donotdelete-pr-sz0vlejvz5iykg',Key='loan-prediction.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


Dealing with the NaN

In [64]:
data.fillna(method='bfill',inplace=True)

Handling the categorical variables

In [65]:
x = pd.Categorical(data['Gender'])              
data['Gender']=x.codes

x = pd.Categorical(data['Married'])              
data['Married']=x.codes

x = pd.Categorical(data['Education'])            
data['Education']=x.codes

x = pd.Categorical(data['Self_Employed'])        
data['Self_Employed']=x.codes

x = pd.Categorical(data['Property_Area'])        
data['Property_Area']=x.codes

x = pd.Categorical(data['Loan_Status'])          
data['Loan_Status'] = x.codes

data['Dependents'] = data['Dependents'].str.replace('+','')     
data['Dependents'] = pd.to_numeric(data['Dependents'])

Creating the independent and the dependent variables

In [66]:
Y = data['Loan_Status']
X = data.drop(columns=['Loan_Status','Loan_ID'])

Using train_test_split for creating the training and the testing datasets

In [67]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.3) 

## k-Nearest Neighbour

In [68]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
clf = KNeighborsClassifier(n_neighbors=5).fit(xtrain, ytrain)
ypred = clf.predict(xtest)
print("Jaccard index: ", metrics.jaccard_similarity_score(ytest, ypred, normalize=True))
print("F1-score: ", metrics.f1_score(ytest, ypred))
print("Log loss: ", metrics.log_loss(ytest, ypred))

Jaccard index:  0.654054054054
F1-score:  0.777777777778
Log loss:  11.9487398463


## Decision Tree

In [69]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=4).fit(xtrain, ytrain)
ypred = clf.predict(xtest)
print("Jaccard index: ", metrics.jaccard_similarity_score(ytest, ypred, normalize=True))
print("F1-score: ", metrics.f1_score(ytest, ypred))
print("Log loss: ", metrics.log_loss(ytest, ypred))

Jaccard index:  0.789189189189
F1-score:  0.864111498258
Log loss:  7.28128144282


## Support Vector Machine

In [70]:
from sklearn.svm import SVC 
clf = SVC().fit(xtrain, ytrain)
ypred = clf.predict(xtest)
print("Jaccard index: ", metrics.jaccard_similarity_score(ytest, ypred, normalize=True))
print("F1-score: ", metrics.f1_score(ytest, ypred))
print("Log loss: ", metrics.log_loss(ytest, ypred))

Jaccard index:  0.718918918919
F1-score:  0.835443037975
Log loss:  9.70842136001


## Logistic Regression

In [71]:
from sklearn.linear_model import LogisticRegression 
clf = LogisticRegression().fit(xtrain, ytrain)
ypred = clf.predict(xtest)
print("Jaccard index: ", metrics.jaccard_similarity_score(ytest, ypred, normalize=True))
print("F1-score: ", metrics.f1_score(ytest, ypred))
print("Log loss: ", metrics.log_loss(ytest, ypred))

Jaccard index:  0.832432432432
F1-score:  0.891986062718
Log loss:  5.78769544526
